In [12]:
# !pip install --quiet transformers==4.5.0
# !pip install --quiet torch
# !pip install --quiet pytorch-lightning==1.2.7
# !pip install termcolor

In [32]:
import torch
import pytorch_lightning as pl
from transformers import (T5ForConditionalGeneration,T5TokenizerFast as T5Tokenizer)

In [33]:
MODEL_NAME="t5-base"
tokenizer=T5Tokenizer.from_pretrained(MODEL_NAME)

In [21]:
class NewsSummaryModel(pl.LightningModule):

  def __init__(self):
    super().__init__()
    self.model=T5ForConditionalGeneration.from_pretrained(MODEL_NAME,return_dict=True)
  def forward(self,input_ids,attention_mask,decoder_attention_mask,labels=None):

    output=self.model(
        input_ids,
        attention_mask=attention_mask,
        labels=labels,
        decoder_attention_mask=decoder_attention_mask
    )
    return output.loss,output.logits

  def training_step(self,batch,batch_idx):
    input_ids=batch["text_input_ids"]
    attention_mask=batch["text_attention_mask"]
    labels=batch["labels"]
    labels_attention_mask=batch["labels_attention_mask"]
    loss,outputs= self(
        input_ids=input_ids,
        attention_mask=attention_mask,
        decoder_attention_mask=labels_attention_mask,
        labels=labels
    )
    self.log("train_loss",loss,prog_bar=True,logger=True)
    return loss

  def test_step(self,batch,batch_idx):
      input_ids=batch["text_input_ids"]
      attention_mask=batch["text_attention_mask"]
      labels=batch["labels"]
      labels_attention_mask=batch["labels_attention_mask"]
      loss,outputs= self(
          input_ids=input_ids,
          attention_mask=attention_mask,
          decoder_attention_mask=labels_attention_mask,
          labels=labels
      )
      self.log("test_loss",loss,prog_bar=True,logger=True)
      return loss

  def configure_optimizers(self):
      return AdamW(self.parameters(),lr=0.0001)

In [43]:
def abs_summarize(text):
  text_encoding=tokenizer(
      text,
      max_length=512,
      padding="max_length",
      truncation=True,
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
  )

  generated_ids=model.model.generate(
      input_ids=text_encoding["input_ids"],
      attention_mask=text_encoding["attention_mask"],
      max_length=150,
      num_beams=2,
      repetition_penalty=2.5,
      length_penalty=1.0,
      early_stopping=True
  )

  preds=[
         tokenizer.decode(gen_id,skip_special_tokens=True,clean_up_tokenization_spaces=True)
         for gen_id in generated_ids
  ]

  return "".join(preds)

In [24]:
text="""Continuing a yearly tradition, outgoing US President Barack Obama signed a proclamation to declare January 16 Religious Freedom Day. "Religious freedom is a principle based not on shared ancestry, culture, ethnicity, or faith but on a shared commitment to liberty - and it lies at the very heart of who we are as Americans," Obama said on Friday.Each year, by Presidential Proclamation, January 16 is declared Religious Freedom Day in the US.Obama urged the country's citizens to reject politics that seeks to target people because of their religion."Part of being American means guarding against bigotry and speaking out on behalf of others, no matter their background or belief - whether they are wearing a hijab or a baseball cap, a yarmulke or a cowboy hat," he said.The president said America's strength comes from its diversity."And we must be unified in our commitment to protecting the freedoms of conscience and religious belief and the freedom to live our lives according to them," he said."Brave men and women of faith have challenged our conscience and brought us closer to our founding ideals, from the abolition of slavery to the expansion of civil rights and workers' rights. And throughout our history, faith communities have helped uphold these values by joining in efforts to help those in need -- rallying in the face of tragedy and providing care or shelter in times of disaster," the US President said.Obama rued that in 2015, nearly 20 percent of hate crime victims in America were targeted because of religious bias."That is unacceptable -- and as Americans, we have an obligation to do better. If we are to defend religious freedom, we must remember that when any religious group is targeted, we all have a responsibility to speak up," he said."""


In [28]:
model = NewsSummaryModel()
model.load_state_dict(torch.load("tests/model_file"))
model.eval()

NewsSummaryModel(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseReluDense(
                (wi): Linear(in_features=768, out_features=3072, bias=False)
                (wo): Linear(in_f

In [42]:
print(text,'\n')
print(abs_summarize(text))

Continuing a yearly tradition, outgoing US President Barack Obama signed a proclamation to declare January 16 Religious Freedom Day. "Religious freedom is a principle based not on shared ancestry, culture, ethnicity, or faith but on a shared commitment to liberty - and it lies at the very heart of who we are as Americans," Obama said on Friday.Each year, by Presidential Proclamation, January 16 is declared Religious Freedom Day in the US.Obama urged the country's citizens to reject politics that seeks to target people because of their religion."Part of being American means guarding against bigotry and speaking out on behalf of others, no matter their background or belief - whether they are wearing a hijab or a baseball cap, a yarmulke or a cowboy hat," he said.The president said America's strength comes from its diversity."And we must be unified in our commitment to protecting the freedoms of conscience and religious belief and the freedom to live our lives according to them," he said.